In [1]:
import os
import gzip
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import cv2
import os
import random
import numpy as np
import cv2
from tqdm import tqdm
from glob import glob
import tifffile as tif
from sklearn.model_selection import train_test_split
import json
from ipynb.fs.full.metrics import *
from sklearn.utils import shuffle
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.models import load_model
from ipynb.fs.full.model import *

from albumentations import (
    PadIfNeeded,
    HorizontalFlip,
    VerticalFlip,
    CenterCrop,
    Crop,
    Compose,
    Transpose,
    RandomRotate90,
    ElasticTransform,
    GridDistortion,
    OpticalDistortion,
    RandomSizedCrop,
    OneOf,
    CLAHE,
    RandomBrightnessContrast,
    RandomGamma,
    HueSaturationValue,
    RGBShift,
    RandomBrightness,
    RandomContrast,
    MotionBlur,
    GaussianBlur,
    GaussNoise,
    ChannelShuffle,
    CoarseDropout
)

Using TensorFlow backend.


In [2]:
def read_data(x, y):
    """ Read the image and mask from the given path. """
    image = cv2.imread(x, cv2.IMREAD_COLOR)
    mask = cv2.imread(y, cv2.IMREAD_COLOR)
    return image, mask

In [3]:
def augment_data(images, masks, save_path, augment=True):
    """ Performing data augmentation. """
    crop_size = (512-64, 512-64)
    size = (512, 512)

    for image, mask in tqdm(zip(images, masks), total=len(images)):
        image_name = image.split("/")[-1].split(".")[0]
        mask_name = mask.split("/")[-1].split(".")[0]

        x, y = read_data(image, mask)
        try:
            h, w, c = x.shape
        except Exception as e:
            image = image[:-1]
            x, y = read_data(image, mask)
            h, w, c = x.shape

        if augment == True:
            ## Center Crop
            aug = CenterCrop(p=1, height=crop_size[0], width=crop_size[1])
            augmented = aug(image=x, mask=y)
            x1 = augmented['image']
            y1 = augmented['mask']

            ## Crop
            x_min = 0
            y_min = 0
            x_max = x_min + size[0]
            y_max = y_min + size[1]

            aug = Crop(p=1, x_min=x_min, x_max=x_max, y_min=y_min, y_max=y_max)
            augmented = aug(image=x, mask=y)
            x2 = augmented['image']
            y2 = augmented['mask']

            ## Random Rotate 90 degree
            aug = RandomRotate90(p=1)
            augmented = aug(image=x, mask=y)
            x3 = augmented['image']
            y3 = augmented['mask']

            ## Transpose
            aug = Transpose(p=1)
            augmented = aug(image=x, mask=y)
            x4 = augmented['image']
            y4 = augmented['mask']

            ## ElasticTransform
            aug = ElasticTransform(p=1, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03)
            augmented = aug(image=x, mask=y)
            x5 = augmented['image']
            y5 = augmented['mask']

            ## Grid Distortion
            aug = GridDistortion(p=1)
            augmented = aug(image=x, mask=y)
            x6 = augmented['image']
            y6 = augmented['mask']

            ## Optical Distortion
            aug = OpticalDistortion(p=1, distort_limit=2, shift_limit=0.5)
            augmented = aug(image=x, mask=y)
            x7 = augmented['image']
            y7 = augmented['mask']

            ## Vertical Flip
            aug = VerticalFlip(p=1)
            augmented = aug(image=x, mask=y)
            x8 = augmented['image']
            y8 = augmented['mask']

            ## Horizontal Flip
            aug = HorizontalFlip(p=1)
            augmented = aug(image=x, mask=y)
            x9 = augmented['image']
            y9 = augmented['mask']

            ## Grayscale
            x10 = cv2.cvtColor(x, cv2.COLOR_RGB2GRAY)
            y10 = y

            ## Grayscale Vertical Flip
            aug = VerticalFlip(p=1)
            augmented = aug(image=x10, mask=y10)
            x11 = augmented['image']
            y11 = augmented['mask']

            ## Grayscale Horizontal Flip
            aug = HorizontalFlip(p=1)
            augmented = aug(image=x10, mask=y10)
            x12 = augmented['image']
            y12 = augmented['mask']

            ## Grayscale Center Crop
            aug = CenterCrop(p=1, height=crop_size[0], width=crop_size[1])
            augmented = aug(image=x10, mask=y10)
            x13 = augmented['image']
            y13 = augmented['mask']

            ##
            aug = RandomBrightnessContrast(p=1)
            augmented = aug(image=x, mask=y)
            x14 = augmented['image']
            y14 = augmented['mask']

            aug = RandomGamma(p=1)
            augmented = aug(image=x, mask=y)
            x15 = augmented['image']
            y15 = augmented['mask']

            aug = HueSaturationValue(p=1)
            augmented = aug(image=x, mask=y)
            x16 = augmented['image']
            y16 = augmented['mask']

            aug = RGBShift(p=1)
            augmented = aug(image=x, mask=y)
            x17 = augmented['image']
            y17 = augmented['mask']

            aug = RandomBrightness(p=1)
            augmented = aug(image=x, mask=y)
            x18 = augmented['image']
            y18 = augmented['mask']

            aug = RandomContrast(p=1)
            augmented = aug(image=x, mask=y)
            x19 = augmented['image']
            y19 = augmented['mask']

            aug = MotionBlur(p=1, blur_limit=7)
            augmented = aug(image=x, mask=y)
            x20 = augmented['image']
            y20 = augmented['mask']

            aug = GaussNoise(p=1)
            augmented = aug(image=x, mask=y)
            x23 = augmented['image']
            y23 = augmented['mask']

            aug = ChannelShuffle(p=1)
            augmented = aug(image=x, mask=y)
            x24 = augmented['image']
            y24 = augmented['mask']

            aug = CoarseDropout(p=1, max_holes=8, max_height=32, max_width=32)
            augmented = aug(image=x, mask=y)
            x25 = augmented['image']
            y25 = augmented['mask']

            images = [
                x, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10,
                x11, x12, x13, x14, x15, x16, x17, x18, x19, x20,
                x23, x24, x25
            ]
            masks  = [
                y, y1, y2, y3, y4, y5, y6, y7, y8, y9, y10,
                y11, y12, y13, y14, y15, y16, y17, y18, y19, y20,
                y23, y24, y25
            ]

        else:
            images = [x]
            masks  = [y]

        idx = 0
        for i, m in zip(images, masks):
            i = cv2.resize(i, size)
            m = cv2.resize(m, size)

            tmp_image_name = f"{image_name}_{idx}.png"
            tmp_mask_name  = f"{mask_name}_{idx}.png"

            image_path = os.path.join(save_path, "image/", tmp_image_name)
            mask_path  = os.path.join(save_path, "mask/", tmp_mask_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            idx += 1

In [4]:
def get_data(path, split=0.1):
    train_x = glob(os.path.join(path, "image_split_trainx/*"))
    train_y = glob(os.path.join(path, "mask_split_trainy/*"))

    valid_x = glob(os.path.join(path, "image_split_validx/*"))
    valid_y = glob(os.path.join(path, "mask_split_validy/*"))

    test_x = glob(os.path.join(path, "image_split_testx/*"))
    test_y = glob(os.path.join(path, "mask_split_testy/*"))

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [5]:
def create_dir(path):
    """ Create a directory. """
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"Error: creating directory with name {path}")

In [6]:
def split_data(path, split=0.1):
    """ Load all the data and then split them into train and valid dataset. """
    size = (512,512)
    img_path = glob(os.path.join(path, "images/*"))
    msk_path = glob(os.path.join(path, "masks/*"))
    save_path = "E:\\Teknofest\\Saglik\\Model1\\"
    
    create_dir("mask_split_trainy/")
    create_dir("image_split_trainx/")
    create_dir("mask_split_testy/")
    create_dir("image_split_testx/")
    create_dir("mask_split_validy/")
    create_dir("image_split_validx/")
    
    img_path.sort()
    msk_path.sort()

    len_ids = len(img_path)
    print(len_ids)
    train_size = int((80/100)*len_ids)
    valid_size = int((10/100)*len_ids)
    test_size = int((10/100)*len_ids)

    train_x, test_x = train_test_split(img_path, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(msk_path, test_size=test_size, random_state=42)

    train_x, valid_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, valid_y = train_test_split(train_y, test_size=test_size, random_state=42)
    
    for i in train_x:
        try:
            image_name = os.path.basename(i)
            i = cv2.imread(i)
            i = cv2.resize(i, size)
            tmp_image_name = f"{image_name}.png"
            image_path = os.path.join(save_path, "image_split_trainx/", tmp_image_name)
            cv2.imwrite(image_path, i)
        except Exception as e:
            print(str(e))
    
    for i in train_y:
        try:
            mask_name = os.path.basename(i)
            i = cv2.imread(i)
            i = cv2.resize(i, size)
            tmp_mask_name = f"{mask_name}.png"
            mask_path = os.path.join(save_path, "mask_split_trainy/", tmp_mask_name)
            cv2.imwrite(mask_path, i)
        except Exception as e:
            print(str(e))
            
    for i in test_x:
        try:
            image_name = os.path.basename(i)
            i = cv2.imread(i)
            i = cv2.resize(i, size)
            tmp_image_name = f"{image_name}.png"
            image_path = os.path.join(save_path, "image_split_testx/", tmp_image_name)
            cv2.imwrite(image_path, i)
        except Exception as e:
            print(str(e))
            
    for i in test_y:
        try:
            mask_name = os.path.basename(i)
            i = cv2.imread(i)
            i = cv2.resize(i, size)
            tmp_mask_name = f"{mask_name}.png"
            mask_path = os.path.join(save_path, "mask_split_testy/", tmp_mask_name)
            cv2.imwrite(mask_path, i)
        except Exception as e:
            print(str(e))
            
    for i in valid_x:
        try:
            image_name = os.path.basename(i)
            i = cv2.imread(i)
            i = cv2.resize(i, size)
            tmp_image_name = f"{image_name}.png"
            image_path = os.path.join(save_path, "image_split_validx/", tmp_image_name)
            cv2.imwrite(image_path, i)
        except Exception as e:
            print(str(e))
            
    for i in valid_y:
        try:
            mask_name = os.path.basename(i)
            i = cv2.imread(i)
            i = cv2.resize(i, size)
            tmp_mask_name = f"{mask_name}.png"
            mask_path = os.path.join(save_path, "mask_split_validy/", tmp_mask_name)
            cv2.imwrite(mask_path, i)
        except Exception as e:
            print(str(e))

In [7]:
def datasett():
    np.random.seed(42)
    split_data("E:\\Teknofest\\Saglik\\Data\\k")
    path = "E:\\Teknofest\\Saglik\\Model1\\"

    train_x = glob(os.path.join(path, "image_split_trainx/*"))
    train_y = glob(os.path.join(path, "mask_split_trainy/*"))

    valid_x = glob(os.path.join(path, "image_split_validx/*"))
    valid_y = glob(os.path.join(path, "mask_split_validy/*"))

    test_x = glob(os.path.join(path, "image_split_testx/*"))
    test_y = glob(os.path.join(path, "mask_split_testy/*"))


    create_dir("augmented_data/train/image/")
    create_dir("augmented_data/train/mask/")
    create_dir("augmented_data/valid/image/")
    create_dir("augmented_data/valid/mask/")
    create_dir("augmented_data/test/image/")
    create_dir("augmented_data/test/mask/")

    augment_data(train_x, train_y, "augmented_data/train/", augment=True)
    augment_data(valid_x, valid_y, "augmented_data/valid/", augment=False)
    augment_data(test_x, test_y, "augmented_data/test/", augment=False)

In [8]:
def read_params():
    """ Reading the parameters from the JSON file."""
    with open("params.json", "r") as f:
        data = f.read()
        params = json.loads(data)
        return params

In [9]:
def load_data(path):
    """ Loading the data from the given path. """
    images_path = os.path.join(path, "image/*")
    masks_path  = os.path.join(path, "mask/*")

    images = glob(images_path)
    masks  = glob(masks_path)

    return images, masks

In [10]:
def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

In [11]:
def load_model_weight(path):
    with CustomObjectScope({
        'dice_loss': dice_loss,
        'dice_coef': dice_coef,
        'bce_dice_loss': bce_dice_loss,
        'focal_loss': focal_loss,
        'iou': iou
        }):
        model = load_model(path)
    return model

Datasett() metodunu ayırdıktan sonra sil yoks train tekrar çalıştırır

In [ ]:
datasett()

6636


 53%|█████████████████████████████████████████▎                                    | 2809/5310 [11:41<10:17,  4.05it/s]